In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
avo=pd.read_csv('avocado.csv')
avo

In [ ]:
avo.info()

In [ ]:
avo=avo.drop(columns=avo.columns.values[0],axis=1)
avo.info()

In [ ]:
avo.Date=avo.Date.astype('datetime64[ns]')
avo.info()

In [ ]:
avo.type.value_counts()

In [ ]:
avo.type=avo.type.map({
    'conventional': 0,
    'organic': 1
    }).astype('int64')
avo.info()

In [ ]:
avo['year']=avo.Date.dt.year
avo['month']=avo.Date.dt.month
avo=avo.drop(columns=['Date'],axis=1)
avo.info()

## Classification to predict the region

In [ ]:
avo_classification=avo.copy()
le=LabelEncoder()
avo_classification.region=le.fit_transform(avo_classification.region)
avo_classification.info()

In [ ]:
X,y=avo_classification.drop(columns=['region'],axis=1),avo_classification.region
X_train,X_test,y_train,y_test=train_test_split(X,
                                               y,
                                               test_size=0.2,
                                               shuffle=True,
                                               stratify=y,
                                               random_state=42)

In [ ]:
scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [ ]:
# Random forest model
rfc=RandomForestClassifier(random_state=42)
cv_score=cross_val_score(estimator=rfc,
                        X=X_train_scaled,
                        y=y_train,
                        scoring='accuracy',
                        cv=10,
                        n_jobs=-1).mean()
rfc.fit(X=X_train_scaled,
        y=y_train)
test_score=rfc.score(X=X_test_scaled,
                     y=y_test)
print(f'Cross val score = {cv_score}')
print(f'Test score = {test_score}')
print(f'Score diff = {abs(cv_score-test_score)}')

In [ ]:
# Gradient boosting model
gbc=GradientBoostingClassifier(random_state=42)
cv_score=cross_val_score(estimator=gbc,
                        X=X_train_scaled,
                        y=y_train,
                        scoring='accuracy',
                        cv=10,
                        n_jobs=-1).mean()
gbc.fit(X=X_train_scaled,
        y=y_train)
test_score=gbc.score(X=X_test_scaled,
                     y=y_test)
print(f'Cross val score = {cv_score}')
print(f'Test score = {test_score}')
print(f'Score diff = {abs(cv_score-test_score)}')

In [ ]:
## Final model is the Random Forest model
feature_importance=pd.Series(data=rfc.feature_importances_,
                             index=X.columns.values).sort_values()
plt.figure(figsize=(10,6))
feature_importance.plot.barh();

In [ ]:
y_pred=rfc.predict(X_test_scaled)
print(classification_report(y_pred=y_pred,
                            y_true=y_test))

## Regression to predict the price

In [ ]:
avo_reg=avo.copy()

In [ ]:
avo_reg=pd.concat([avo_reg.drop(columns=['region'],axis=1),pd.get_dummies(avo.region,
                                  drop_first=True).astype('int64')], 
                                  axis=1,
                                  join='inner')
avo_reg

In [ ]:
X,y=avo_reg.drop(columns=['AveragePrice'],axis=1),avo_reg.AveragePrice

X_train,X_test,y_train,y_test=train_test_split(X,
                                               y,
                                               test_size=0.2,
                                               shuffle=True,
                                               random_state=42)
scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [ ]:
# Random forest model
rfr=RandomForestRegressor(random_state=42)
cv_score=cross_val_score(estimator=rfr,
                         X=X_train_scaled,
                         y=y_train,
                         scoring='neg_mean_absolute_error',
                         cv=10,
                         n_jobs=-1).mean()*-1
rfr.fit(X=X_train_scaled,
        y=y_train)
test_score=mean_absolute_error(y_pred=rfr.predict(X=X_test_scaled),
                               y_true=y_test)
print(f'Cross val score = {cv_score}')
print(f'Test score = {test_score}')
print(f'Score diff = {abs(cv_score-test_score)}')

In [ ]:
# Gradient boosting model
gbr=GradientBoostingRegressor(random_state=42)
cv_score=cross_val_score(estimator=gbr,
                         X=X_train_scaled,
                         y=y_train,
                         scoring='neg_mean_absolute_error',
                         cv=10,
                         n_jobs=-1).mean()*-1
gbr.fit(X=X_train_scaled,
        y=y_train)
test_score=mean_absolute_error(y_pred=gbr.predict(X=X_test_scaled),
                               y_true=y_test)
print(f'Cross val score = {cv_score}')
print(f'Test score = {test_score}')
print(f'Score diff = {abs(cv_score-test_score)}')

In [ ]:
## Final regression model is the Random Forest

feature_importance=pd.Series(data=rfr.feature_importances_,
                             index=X.columns.values).sort_values(ascending=False)
plt.figure(figsize=(15,6))
feature_importance.plot.bar();
y_pred=rfr.predict(X_test_scaled)

In [ ]:
selected_features=feature_importance.cumsum()[feature_importance.cumsum()<=0.95].index.values
selected_features

In [ ]:
X=X.copy()[selected_features]
X_train,X_test,y_train,y_test=train_test_split(X,
                                               y,
                                               test_size=0.2,
                                               shuffle=True,
                                               random_state=42)

scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [ ]:
# Random forest model
rfr=RandomForestRegressor(random_state=42)
cv_score=cross_val_score(estimator=rfr,
                         X=X_train_scaled,
                         y=y_train,
                         scoring='neg_mean_absolute_error',
                         cv=10,
                         n_jobs=-1).mean()*-1
rfr.fit(X=X_train_scaled,
        y=y_train)
test_score=mean_absolute_error(y_pred=rfr.predict(X=X_test_scaled),
                               y_true=y_test)
print(f'Cross val score = {cv_score}')
print(f'Test score = {test_score}')
print(f'Score diff = {abs(cv_score-test_score)}')

In [ ]:
feature_importance=pd.Series(data=rfr.feature_importances_,
                             index=X.columns.values).sort_values(ascending=False)
plt.figure(figsize=(12,6))
feature_importance.plot.bar();
y_pred=rfr.predict(X_test_scaled)